<a href="https://colab.research.google.com/github/18BIS036/hpefinal/blob/main/hpefinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
directory='/content/forms'

TensorFlow 1.x selected.
1.15.2


In [2]:
def image_binarization(image):
  # converting image to grayscale
  image_grayscale=image.convert('L')
  img=np.array(image_grayscale) 
   
  img[img <128] = 0
  img[img >=128] = 254 

  #making white as foreground pixels and black as background pixels
  img[img==0]=255
  img[img==254]=0
  
  return img


def line_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a line
  
  lines=[]
  begin_matrix=[]
  stop_matrix=[]
  del_start_matrix=[]
  del_end_matrix=[]

  horizontal_hist = np.sum(img,axis=1,keepdims=True)/255
  start_count=0

  for i in range(len(horizontal_hist)):

    if horizontal_hist[i]>0 and horizontal_hist[i-1]==0:
      start_count+=1
      start_matrix.append(i)

    if horizontal_hist[i]==0 and start_count>0 and horizontal_hist[i-1]>0:
      end_matrix.append(i)

  
  
  if len(start_matrix)==len(end_matrix):
    for i in range(len(start_matrix)):
      if end_matrix[i]-start_matrix[i]<20:
        del_start_matrix.append(i)
        del_end_matrix.append(i)
    for i in range(len(start_matrix)):
      count=0
      for j in range(len(del_start_matrix)):
        if i==del_start_matrix[j]:
          count=count+1
      if count==0:
        begin_matrix.append(start_matrix[i])
    for i in range(len(end_matrix)):
      count=0
      for j in range(len(del_end_matrix)):
        if i==del_end_matrix[j]:
          count=count+1
      if count==0:
        stop_matrix.append(end_matrix[i])
    
    for i in range(len(begin_matrix)):
      lines.append(img[begin_matrix[i]:stop_matrix[i],:])

      

      

  
  return lines




def word_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a word
  dissection_matrix=[]
  
  words=[]
  m,n=img.shape
  
  length=[]
  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  
  
  start_count=0
  
  
  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]>0 and vertical_hist[0][i-1]==0:
      start_count+=1
      start_matrix.append(i)
    if vertical_hist[0][i]==0 and start_count>0 and vertical_hist[0][i-1]>0:
      end_matrix.append(i)
  
  
  length_mag=0
  for i in range(len(start_matrix)):
    if i>0:
      length_mag=(start_matrix[i]-end_matrix[i-1])
      length.append(length_mag)

  max=np.max(length)
     
  avg=max/3

  dissection_matrix.append([start_matrix[0],end_matrix[0]])
  j=0
  for i in range(len(length)-1):
    
    if length[i]> avg:
      dissection_matrix.append([start_matrix[i+1],end_matrix[i+1]])
      j=j+1
      
    if length[i]<=avg:
      dissection_matrix[j][1]=end_matrix[i+1]
  for i in range(len(dissection_matrix)):
    words.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
  
  return words


def char_segmentation(img):
  start_matrix=[]
  dissection_matrix=[]
  delete_matrix=[]
  address_matrix=[]
  
  
  characters=[]
  m,n=img.shape

  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  start_matrix.append(0)

  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]<10:
      start_matrix.append(i)

  for i in range(len(start_matrix)-1):
    if start_matrix[i+1]-start_matrix[i]<10:
      delete_matrix.append(i)
  
  for i in range(len(start_matrix)):
    count=0
    for j in range(len(delete_matrix)):
      if  i==delete_matrix[j]:
        count=count+1
    if count==0:
      address_matrix.append(start_matrix[i])


  for i in range(len(address_matrix)-1):
    dissection_matrix.append([address_matrix[i],address_matrix[i+1]])
  for i in range(len(dissection_matrix)):
    characters.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
    
    
  return characters

In [3]:
def data_label():

  samples=[]
  labels=[]

 
  
 
  
  for filename in os.listdir(directory):
    count=0
    text=open('/content/lines_new.txt')
    image_orig=Image.open('/content/forms/'+filename,'r')
    img=image_binarization(image_orig)
    lines=line_segmentation(img)
    for line in text:
      line_Split = line.strip().split(" ")
      filename_split=filename.split('.')
      linesplit=line_Split[0].split("-")
      string= linesplit[0]+'-'+linesplit[1]
      if string==filename_split[0]:
          count=count+1
    if count==len(lines): 
      for i in range(len(lines)):
        try:
          words=word_segmentation(lines[i])
          text=open('/content/lines_new.txt')
          for line in text:
            lineSplit = line.strip().split(' ')
            filename_split=filename.split('.')
            if lineSplit[0]==filename_split[0]+'-'+str(0)+str(i):
              word_split = lineSplit[8].split('|')
              if len(word_split)==len(words):
                for j in range(len(words)):
                  characters=char_segmentation(words[j])
                  if len(word_split[j])==len(characters):
                    for k in range(len(characters)):
                      
                      character=cv2.resize(characters[k],(28,28))
                      samples.append(character)
                    for l in word_split[j]:
                      labels.append(l) 

        except ValueError as ve:
          continue  
  
                 
  
  return  samples,labels

samples,labels=data_label()
print(len(samples))
print(len(labels))
for i in range(5):
  print(labels[i])

224
224
a
n
d
M
r


In [4]:
samples = np.array(samples, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels
samples.shape

(224, 28, 28)

In [5]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(samples, labels, test_size=0.2, random_state=42)


In [6]:
len(testx)
testx.shape
trainx=trainx.reshape(trainx.shape[0],28,28,1)
testx=testx.reshape(testx.shape[0],28,28,1)


In [7]:
HP_epoch = 1000
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dims = (28,28,1)
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [8]:
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'

In [9]:
aug = ImageDataGenerator( rotation_range=25, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [10]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.25
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 

HP_img_dims = (28,28,1)

In [11]:
class RacoonVGG:
  @staticmethod
  def build(height, width,depth, classes):
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model


trainx=trainx.reshape(trainx.shape[0],28,28,1)
testx=testx.reshape(testx.shape[0],28,28,1)


In [12]:
import keras
import keras.utils
from keras import utils as np_utils
from keras.optimizers import adam
model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1],depth=HP_img_dims[2],classes = len(lb.classes_))
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
hist = model.fit(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                 validation_data=(testx, testy),
                 steps_per_epoch= len(trainx) // HP_batch_size,
                 epochs=HP_epoch, verbose=1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Epoch 1/1000
5/5 [==============================] - 3s 515ms/step - loss: 5.1292 - accuracy: 0.0272 - val_loss: 3.3561 - val_accuracy: 0.0444
Epoch 2/1000
5/5 [==============================] - 1s 191ms/step - loss: 4.5905 - accuracy: 0.0476 - val_loss: 3.3371 - val_accuracy: 0.0444
Epoch 3/1000
5/5 [==============================] - 1s 193ms/step - loss: 4.5816 - accuracy: 0.0884 - val_loss: 3.3225 - val_accuracy: 0.0444
Epoch 4/1000
5/5 [==============================] - 1s 207ms/step - loss: 4.2644 - accuracy: 0.0875 - val_loss: 3.3084 - val_accuracy: 0.0444
Epoch 5/1000
5/5 [==============================] - 1s 188ms/step - loss: 4.2809 - accuracy: 0.0952 - val_loss: 3.3007 - val_accuracy: 0.0444
Epoch 6/1000
5/5 [==============================] - 1s 198ms/step - loss: 4.1948 - accuracy: 0.0748 - val_loss: 3.2949 - val_accuracy: 0.0444
Epoch 7/1000
5/5 [==============================] - 1s 182ms/step

In [37]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=4a7c173a54a48027a9afea549b0dc87afcd1ab8de6c1a25e8fcd8a8d46ac85b5
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [45]:
pip install hyperopt

In [96]:
pip install scikit-optimize

     |████████████████████████████████| 102kB 3.6MB/s 


In [97]:
# import packages
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import StandardScaler
from skopt.searchcv import BayesSearchCV
from skopt.space import Integer, Real, Categorical 
from skopt.utils import use_named_args
from skopt import gp_minimize

import warnings

warnings.filterwarnings("ignore")

In [98]:
# define search space 
params = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": (1, 9),
    "criterion": ["gini", "entropy"],
}

In [102]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
rf_classifier = Sequential()
search = BayesSearchCV(
    estimator=rf_classifier,
    search_spaces=params,
    n_jobs=1,
    cv=5,
    n_iter=30,
    scoring="accuracy",
    verbose=4,
    random_state=42
)

In [120]:
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(X,Y,test_size)
search.fit(x_train,y_test)

NameError: ignored

In [121]:
# report the best result

print(search.best_score_)
print(search.best_params_)

TypeError: ignored